# Loop 6 Analysis: Submission Failure Investigation

The corner extraction experiment (006) failed with 'Overlapping trees in group 040'. Analysis shows the issue was precision truncation during the corner extraction process.

In [ ]:
import pandas as pd
import numpy as np

# Compare precision between candidate_004 and candidate_005
prev = pd.read_csv('/home/code/submission_candidates/candidate_004.csv')
curr = pd.read_csv('/home/code/submission_candidates/candidate_005.csv')

# Check N=40 precision
prev_40 = prev[prev['id'].str.startswith('040_')]
curr_40 = curr[curr['id'].str.startswith('040_')]

print('Previous N=40 first tree:')
print(f"  x: {prev_40.iloc[0]['x']}")
print(f"  y: {prev_40.iloc[0]['y']}")
print(f"  deg: {prev_40.iloc[0]['deg']}")

print('\nCurrent N=40 first tree:')
print(f"  x: {curr_40.iloc[0]['x']}")
print(f"  y: {curr_40.iloc[0]['y']}")
print(f"  deg: {curr_40.iloc[0]['deg']}")

print('\nPrecision difference:')
print(f"  x: {len(prev_40.iloc[0]['x'])} -> {len(curr_40.iloc[0]['x'])} chars")
print(f"  y: {len(prev_40.iloc[0]['y'])} -> {len(curr_40.iloc[0]['y'])} chars")
print(f"  deg: {len(prev_40.iloc[0]['deg'])} -> {len(curr_40.iloc[0]['deg'])} chars")

## Key Finding

The corner extraction code used numpy float64 which truncated precision from ~25 decimal places to ~16 decimal places. This precision loss caused trees that were previously just touching to now overlap.

## jiweiliu Kernel Analysis

The jiweiliu kernel has two key techniques:
1. **Grid-based initialization with SA** - Generate configurations from grid patterns
2. **Deletion cascade** - Propagate good large configs to smaller sizes by iteratively removing trees

The deletion cascade is the most promising technique for our current situation.

In [ ]:
# Read the jiweiliu kernel to understand deletion cascade
with open('/home/code/research/kernels/jiweiliu_super-fast-simulated-annealing-with-translations/super-fast-simulated-annealing-with-translations.ipynb', 'r') as f:
    content = f.read()

# Find the deletion cascade function
import json
notebook = json.loads(content)
for cell in notebook['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source']) if isinstance(cell['source'], list) else cell['source']
        if 'deletion_cascade' in source:
            print('Found deletion cascade code:')
            print(source[:2000])
            break

## Strategy for Next Experiment

1. **Implement jiweiliu deletion cascade** - This technique iteratively removes trees from large configurations to generate smaller ones
2. **Preserve precision** - Use string operations to preserve the original precision
3. **Grid-based SA** - Generate new configurations from grid patterns

The deletion cascade is particularly promising because:
- It generates NOVEL configurations (different from existing ones)
- It's fast (~3 seconds according to the kernel)
- It claims ~0.15 improvement